# Greenplum Database  Concepts Explained - Part 1
## 1. System Setup
### 1.1 Initialize database connection and setup global variable values

In [4]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql

In [5]:
%sql $CONNECTION_STRING
%sql $DB_USER@$DB_NAME {"SELECT version();"}

In [6]:
query = "SHOW gp_autostats_mode; \
ALTER DATABASE {} SET gp_autostats_mode TO 'NONE'; \
SHOW gp_autostats_mode;".format(DB_NAME)

%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
on_no_stats


In [7]:
%%sql $DB_USER@$DB_NAME
SELECT version();

1 rows affected.


version
"PostgreSQL 9.4.24 (Greenplum Database 6.6.0 build commit:12ef7352da41898153e3c7049c5f5c32841239d4) on x86_64-unknown-linux-gnu, compiled by gcc (GCC) 6.4.0, 64-bit compiled on Apr 7 2020 02:46:13"


## 2. The Amazon Customer Reviews Dataset

Over 130+ million customer reviews are available to researchers as part of this release. The data is available in TSV files in the `amazon-reviews-pds` S3 bucket in AWS US East Region. Each line in the data files corresponds to an individual review (tab delimited, with no quote and escape characters). Samples of the data are available in English and French; more details on the information in each column can be found [here](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt).

If you use the AWS Command Line Interface, you can list data in the bucket with the `aws s3 ls` command:

`aws s3 ls s3://amazon-reviews-pds/tsv/`

To download data using the AWS Command Line Interface, you can use the `aws s3 cp` command. For instance, the following command will copy the file named `amazon_reviews_us_Camera_v1_00.tsv.gz` to your local directory:

`aws s3 cp s3://amazon-reviews-pds/tsv/<S3 File> <Local File>`

### 2.1 Prepare AWS System and setup awscli library via pip

In [8]:
shfilecode = !pygmentize -f html -O full,style=friendly -l shell script/1-1-system-prepare.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 cd $HOME 

curl -O https://bootstrap.pypa.io/get-pip.py
python $HOME /get-pip.py --force-reinstall
rm $HOME /get-pip.py

mv $HOME /.bash_profile $HOME /.bash_profile.old

cat >> $HOME /.bash_profile << 'EOF' 
 # .bash_profile 

 # Get the aliases and functions 
 if [ -f ~/.bashrc ]; then 
 . ~/.bashrc 
 fi 

 # User specific environment and startup programs 

 export PATH=$HOME/.local/bin:$HOME/bin:$PATH 

 EOF 

chmod 644 $HOME /.bash_profile
 source $HOME /.bash_profile

pip install --upgrade --force-reinstall pip
pip install --upgrade --force-reinstall boto3
pip install --upgrade --force-reinstall awscli
pip install --upgrade --force-reinstall matplotlib

 echo "***********************" 
 echo "* Get AWS CLI version *" 
 echo "***********************" 
aws --version

In [9]:
!ssh-keygen -R $DB_SERVER
!ssh-keyscan $DB_SERVER >> ~/.ssh/known_hosts
!scp -i ~/.ssh/aws-gp.pem script/1-1-system-prepare.sh $DB_USER@$DB_SERVER:system-prepare.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./system-prepare.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./system-prepare.sh'

# Host ec2-3-10-217-75.eu-west-2.compute.amazonaws.com found: line 11
/root/.ssh/known_hosts updated.
Original contents retained as /root/.ssh/known_hosts.old
# ec2-3-10-217-75.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-3-10-217-75.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-3-10-217-75.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
1-1-system-prepare.sh                         100%  763    69.6KB/s   00:00    
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1825k  100 1825k    0     0  9798k      0 --:--:-- --:--:-- --:--:-- 9761k
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/rele

***********************
* Get AWS CLI version *
***********************
aws-cli/1.15.30 Python/2.7.5 Linux/3.10.0-1062.18.1.el7.x86_64 botocore/1.10.30


### 2.2 Provide AWS Access Key ID & Secret Access Key

In [10]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-2-aws-configure.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 if [ -d $HOME /.aws ] ; then rm -rf $HOME /.aws ; fi 

mkdir $HOME /.aws

 echo "[default] 
 aws_access_key_id=" $1 " 
 aws_secret_access_key=" $2 "" > $HOME /.aws/credentials

 echo "[default] 
 region=eu-west-2 
 output=json" > $HOME /.aws/config

 #test aws config 
 if [ " $( /usr/local/bin/aws s3 ls s3://amazon-reviews-pds/tsv/ | awk '{print $4}' | awk 'END {print NR}' ) " -eq "0" ] ; 
 then echo "Error on AWS S3 Configuration setup" ; 
 else echo "AWS S3 Configuration setup correctly" ; 
 fi

In [11]:
import getpass

!scp -i ~/.ssh/aws-gp.pem script/1-2-aws-configure.sh $DB_USER@$DB_SERVER:aws-configure.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-configure.sh'

cmd = 'sudo ./aws-configure.sh ' 
cmd = cmd + getpass.getpass("AWS Access Key ID [None]:") 
cmd = cmd + ' ' + getpass.getpass("AWS Secret Access Key [None]:")

!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

1-2-aws-configure.sh                          100%  484    52.2KB/s   00:00    
AWS Access Key ID [None]:········
AWS Secret Access Key [None]:········
AWS S3 Configuration setup correctly


### 2.3 Copy source files from AWS S3
For our demo, we choose to download the available files into the `/home/gpadmin/data/` folder, using the `aws s3 cp <S3 File> <Local File>` command described before, as follows:

In [12]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-3-aws-s3-copy.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 export S3_DATA_DIR = /data1/tmp_s3_data

 if [ -d $S3_DATA_DIR ] ; then rm -rf $S3_DATA_DIR ; fi 

mkdir $S3_DATA_DIR 
 cd $S3_DATA_DIR 

ls -laF $S3_DATA_DIR 

/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz amazon_reviews_us_Wireless_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz amazon_reviews_us_Watches_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz amazon_reviews_us_Video_Games_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz amazon_reviews_us_Video_DVD_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz amazon_reviews_us_Video_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz amazon_reviews_us_Toys_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz amazon_reviews_us_Tools_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz amazon_reviews_us_Sports_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz amazon_reviews_us_Software_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz amazon_reviews_us_Shoes_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz amazon_reviews_us_Pet_Products_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz amazon_reviews_us_PC_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz amazon_reviews_us_Outdoors_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz amazon_reviews_us_Office_Products_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz amazon_reviews_us_Music_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz amazon_reviews_us_Major_Appliances_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz amazon_reviews_us_Luggage_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz amazon_reviews_us_Kitchen_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz amazon_reviews_us_Jewelry_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz amazon_reviews_us_Home_Improvement_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz
/usr/

In [13]:
!scp -i ~/.ssh/aws-gp.pem script/1-3-aws-s3-copy.sh $DB_USER@$DB_SERVER:aws-s3-copy.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-s3-copy.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./aws-s3-copy.sh'

1-3-aws-s3-copy.sh                            100% 7251   365.3KB/s   00:00    
total 0
drwxr-xr-x 2 root    root     6 Jun 21 11:55 ./
drwxr-xr-x 4 gpadmin gpadmin 39 Jun 21 11:55 ../
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz to ./amazon_reviews_us_Wireless_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz to ./amazon_reviews_us_Watches_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz to ./amazon_reviews_us_Video_Games_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz to ./amazon_reviews_us_Video_DVD_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz to ./amazon_reviews_us_Video_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz to ./amazon_reviews_us_Toys_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz to ./amazo

## 3. Data Loading
### 3.1. Create the Schema (optional) and the Database Table to hold the dataset, as shown below:

In [14]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-1-create-db-schema-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP SCHEMA IF EXISTS demo CASCADE ; 

 CREATE SCHEMA demo ; 

 DROP TABLE IF EXISTS demo . amzn_reviews ; 

 CREATE TABLE demo . amzn_reviews ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id );

In [15]:
query = !cat script/2-1-create-db-schema-table.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

Done.
Done.
Done.
Done.


[]

In [16]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-2-count-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [17]:
query = !cat script/2-2-count-table.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.


count
0


### 3.2. Load the Input Dataset using the gpload Utility
**gpload** is a data loading utility that acts as an interface to the Greenplum Database external table parallel loading feature. Using a load specification defined in a YAML formatted control file, gpload executes a load by invoking the Greenplum Database parallel file server (**gpfdist**), creating an external table definition based on the source data defined, and executing an *INSERT*, *UPDATE* or *MERGE* operation to load the source data into the target table in the database.

You can declare more than one file as input/source as long as the data is of the same format in all files specified. Additionally, if the files are compressed using **gzip** or **bzip2** (have a .gz or .bz2 file extension), the files will be uncompressed automatically (provided that gunzip or bunzip2 is in your path). You can also declare options such as the schema of the source data files, perform basic transformations, define custom delimiter and/or escape character(s), and many more. For the full list of available options, check the GPLoad Utility Reference available on [Pivotal Greenplum Database Documentation](https://gpdb.docs.pivotal.io/latest) (*Pivotal Greenplum Documentation > Utility Guide > Management Utility Reference > gpload*).

The operation, including any SQL commands specified in the SQL collection of the YAML control file, are performed as a single transaction to prevent inconsistent data when performing multiple, simultaneous load operations on a target table.

For our demo, we have prepared the *gpload_amzn_reviews.yaml* YAML control file, as shown here:

In [18]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l yaml script/3-2-gpload-amzn-reviews.yaml
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 VERSION : 1.0.0.1 
 GPLOAD : 
 INPUT : 
 - SOURCE : 
 FILE : 
 - /data1/tmp_s3_data/amazon_reviews_us*.tsv.gz 
 - FORMAT : text 
 - HEADER : true 
 - LOG_ERRORS : true 
 - MAX_LINE_LENGTH : 100000 
 - ERROR_LIMIT : 50000 
 OUTPUT : 
 - TABLE : demo.amzn_reviews 
 - MODE : insert 
 PRELOAD : 
 - TRUNCATE : true 
 - REUSE_TABLES : true

#### 3.2.1. Delete error log information for existing tables in the current database.

In [19]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-1-delete-error-log-info.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT gp_truncate_error_log ( 'demo.amzn_reviews' );

In [20]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload_amzn_reviews.log ]; then rm ./gpload_amzn_reviews.log; fi'

query = !cat script/3-1-delete-error-log-info.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.


gp_truncate_error_log
True


#### 3.2.2. Copy GPLoad YAML file across to the Database Server and execute

In [21]:
!scp -i ~/.ssh/aws-gp.pem script/3-2-gpload-amzn-reviews.yaml $DB_USER@$DB_SERVER:gpload_amzn_reviews.yaml

cmd = "gpload -d {0} -f ./gpload_amzn_reviews.yaml -l ./gpload_amzn_reviews.log 2>&1".format(DB_NAME) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

3-2-gpload-amzn-reviews.yaml                  100%  356    22.8KB/s   00:00    
2020-06-21 12:01:09|INFO|gpload session started 2020-06-21 12:01:09
2020-06-21 12:01:09|INFO|no host supplied, defaulting to localhost
2020-06-21 12:01:09|INFO|started gpfdist -p 8000 -P 9000 -f "/data1/tmp_s3_data/amazon_reviews_us*.tsv.gz" -t 30 -m 100000
2020-06-21 12:01:09|INFO|did not find an external table to reuse. creating ext_gpload_reusable_83128546_b3ae_11ea_b57e_06a1a35f08b6
2020-06-21 12:10:20|WARN|7622 bad rows
2020-06-21 12:10:20|WARN|Please use following query to access the detailed error
2020-06-21 12:10:20|WARN|select * from gp_read_error_log('ext_gpload_reusable_83128546_b3ae_11ea_b57e_06a1a35f08b6') where cmdtime > to_timestamp('1592737269.29')
2020-06-21 12:10:20|INFO|running time: 551.53 seconds
2020-06-21 12:10:20|INFO|rows Inserted          = 150955707
2020-06-21 12:10:20|INFO|rows Updated           = 0
2020-06-21 12:10:20|INFO|data formatting errors = 7622
2020-06-21 12:10:20|INFO|g

### 3.3. Check gpload execution

Check **gpload** execution output (shown above and also available on *./gpload_amzn_reviews.log*), confirm successful loading of the data and/or identify any message which require ones attention and/or actions:

#### 3.3.1. Check the data has been properly loaded, by confirming row count shown above:

In [22]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-3-count-amzn-reviews.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [23]:
query = !cat script/3-3-count-amzn-reviews.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


count
150955707


In [24]:
cmd = 'cat /home/gpadmin/gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|";OFS=" "} {print $3}'"'"'\
    | awk '"'"'{print $1, "COUNT(*)", $3, $4, $5, $6, $7, $8}'"'"''
query = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd
%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.


count
7622


### Continue to Part 2 of Greenplum Database Concepts Explained; [Basic Table Functions](AWS-GP-demo-2.ipynb).